In [1]:
import torch
import numpy as np
import random
import pandas as pd
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import vllm
from vllm import LLM, SamplingParams

2025-09-21 13:41:07.174860: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-21 13:41:07.217381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-21 13:41:07.217419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-21 13:41:07.218816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-21 13:41:07.225603: I tensorflow/core/platform/cpu_feature_guar

INFO 09-21 13:41:08 [__init__.py:216] Automatically detected platform cuda.


In [2]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=56)

In [3]:
llm_model_pth = 'nvidia/OpenReasoning-Nemotron-7B'

llm = LLM(
    llm_model_pth,
    max_num_seqs=64,             
    max_model_len=32768,         
    trust_remote_code=True,       
    tensor_parallel_size=1,   
    seed=56,
)

tokenizer = llm.get_tokenizer()

INFO 09-21 13:41:49 [utils.py:328] non-default args: {'trust_remote_code': True, 'seed': 56, 'max_model_len': 32768, 'max_num_seqs': 64, 'disable_log_stats': True, 'model': 'nvidia/OpenReasoning-Nemotron-7B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

INFO 09-21 13:41:57 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(EngineCore_DP0 pid=1647) INFO 09-21 13:42:45 [default_loader.py:268] Loading weights took 2.66 seconds
(EngineCore_DP0 pid=1647) INFO 09-21 13:42:45 [gpu_model_runner.py:2392] Model loading took 14.2717 GiB and 42.552028 seconds
(EngineCore_DP0 pid=1647) INFO 09-21 13:42:49 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/377aff7576/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=1647) INFO 09-21 13:42:49 [backends.py:550] Dynamo bytecode transform time: 3.81 s
(EngineCore_DP0 pid=1647) INFO 09-21 13:42:52 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:08 [backends.py:215] Compiling a graph for dynamic shape takes 17.91 s
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:11 [monitor.py:34] torch.compile takes 21.72 s in total
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:13 [gpu_worker.py:298] Available KV cache memory: 55.96 GiB
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:13 [kv_cache_util

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 19/19 [00:00<00:00, 32.36it/s]


(EngineCore_DP0 pid=1647) INFO 09-21 13:43:15 [gpu_model_runner.py:3118] Graph capturing finished in 2 secs, took 0.19 GiB
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:15 [gpu_worker.py:391] Free memory on device (78.83/79.32 GiB) on startup. Desired GPU memory utilization is (0.9, 71.39 GiB). Actual usage is 14.27 GiB for weight, 1.13 GiB for peak activation, 0.02 GiB for non-torch memory, and 0.19 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=59728225689` to fit into requested memory, or `--kv-cache-memory=67724004352` to fully utilize gpu memory. Current kv cache memory in use is 60091032985 bytes.
(EngineCore_DP0 pid=1647) INFO 09-21 13:43:15 [core.py:218] init engine (profile, create kv cache, warmup model) took 29.31 seconds
INFO 09-21 13:43:16 [llm.py:295] Supported_tasks: ['generate']
INFO 09-21 13:43:16 [__init__.py:36] No IOProcessor plugins requested by the model


In [4]:
sampling_params = SamplingParams(
    temperature=0.6,              
    n=16,            
    top_p=0.95,                    
    min_p=0.05,                    
    skip_special_tokens=True,   
    max_tokens=32768,             
    seed=56,
)

In [5]:
def create_messages(task_text):
    sys_prompt = """
    You should think step-by-step. Return final answer within \\boxed{}. The final answer should not be in the format LaTeX.
    """

    messages = [
        {
            "role": "system",
            "content": sys_prompt,
        },
        {
            "role": "user",
            "content": f"Task: {task_text}",
        }
    ]

    list_of_texts = tokenizer.apply_chat_template(
        conversation=messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return list_of_texts

In [6]:
df = pd.read_csv("final_submission.csv")
messages = df['task'].apply(create_messages).tolist()

In [ ]:
bs = 16
for i in range(0,len(df),bs):
    messages_batch = messages[i:i+bs]
    results = llm.generate(prompts=messages_batch, sampling_params=sampling_params)
    answers = [[o.text for o in r.outputs] for r in results]
    sub = pd.DataFrame()
    sub['answers'] = answers
    sub['tasks'] = df['task'][i:i+bs].tolist()
    sub.to_parquet(f'sub_nemo_{i}.pqt',index=False)

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/256 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/256 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
results = llm.generate(prompts=messages, sampling_params=sampling_params)
answers = [[o.text for o in r.outputs] for r in results]

In [ ]:
answers = [[o.text for o in r.outputs] for r in results]

In [23]:
df['answer'] = answers
df.to_parquet('predicts_qwen.pqt',index=False)